# Proc tabulate

Służy do tworzenia złożonych zestawień tabelarycznych - zawiera w sobie opcje procedury `means` oraz `freq`.

```
proc tabulate data=zbior <out=zbior_wynikowy> <missing>;
class <lista zmiennych klasyfikujących>;
var <lista zmiennych analizowanych>;
table <wyrażenie_stron>, <wyrażenie_wierszy> , wyrażenie_kolumn 	</lista opcji tabeli>;
by <notsorted> <descending> zmienna_1 < ... <descending> zmienna_n>;
run;
```

Opcje poszczególnych instrukcji:

**CLASS**

- `ascending/descending` - porządek sortowania
- `missing` - wyświetlanie braków danych
- `order=data/formatted/freq/unformatted` - porządek wyświetlania

**TABLE**

- `box=` - tekst dla pustego pola powyżej wierszy

Statystyki poszczególnych instrukcji:

**CLASS**

- `n`
- `nmiss`
- `pctn`

**VAR**

- takie same jak w `proc means`

Operatory

| operator | działanie |
|----------|-----------|
| przecinek | oddziela wymiary w tabeli |
| gwiazdka | przecina elementy wewnątrz wymiaru |
| spacja | łączy elementy wewnątrz wymiaru |
| nawiasy okrągłe | grupuje elementy, aby inny operator mógł być użyty dla całej grupy |
| znak równości | przypisuje etykietę do zmiennej lub statystyki |

In [1]:
libname bib "/folders/myfolders/przetwarzanie-danych-SAS/";

SAS Connection established. Subprocess id is 3297



## Tablice jednowymiarowe

W `proc tabulate` jako argument `class` podajemy zmienne, które będziemy traktować jako zmienne klasyfikujące lub liczyli dla nich częstości, natomiast w argumencie `var` te zmienne, dla których będą wyznaczane statystyki opisowe. W tych dwóch argumentach jedynie te zmienne deklarujemy, natomiast wygląd tabeli ustalamy w argumencie `table`.

In [4]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table wykszt wynik hipoteka;
run;

Domyślną statystyką dla cech w argumencie `class` jest liczebność - `n`. Mamy także do dyspozycji wartość `all` wyświetlającą sumę:

In [5]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table all wykszt wynik hipoteka;
run;

Użycie gwiazdki oraz nawiasów:

In [10]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table all wykszt*(wynik hipoteka);
run;

Liczebności możemy także zamienić na procenty:

In [12]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table (all wykszt*(wynik hipoteka))*pctn="%";
run;

Zobaczmy jak wyglądają miary tendencji centralnej salda w ramach posiadania kredytu hipotecznego

In [14]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table hipoteka*saldo;
run;

Domyślną statystyką dla zmiennych w argumencie `var` jest suma, ale możemy to zmienić.

In [15]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table hipoteka*saldo*mean;
run;

Dodajmy jeszcze medianę:

In [17]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table hipoteka*saldo*(mean median);
run;

I odpowiednie etykiety

In [20]:
proc tabulate data=bib.bank;
class wykszt wynik hipoteka;
var saldo;
table hipoteka="Kredyt hipoteczny"*saldo=""*(mean="Średnie saldo" median="Mediana salda");
run;

Taką tabelę możemy także zapisać do zbioru SAS poprzez argument `out=`.

In [21]:
proc tabulate data=bib.bank out=bank_saldo_hipoteka;
class wykszt wynik hipoteka;
var saldo;
table hipoteka="Kredyt hipoteczny"*saldo=""*(mean="Średnie saldo" median="Mediana salda");
run;

## Tablice dwuwymiarowe

Użycie przecinka w argumencie `table` powoduje utworzenie tablicy krzyżowej.

In [22]:
proc tabulate data=bib.bank;
class wykszt wynik_poprz;
table wykszt, wynik_poprz;
run;

Możemy także globalnie dodać opcję `missing`, która odpowiada za wyświetlanie braków danych. Natomiast opcja `descending` w argumencie `class` zmienia kierunek sortowania kategorii.

In [24]:
proc tabulate data=bib.bank missing;
class wykszt wynik_poprz / descending;
table wykszt, wynik_poprz;
run;

Dodajmy podsumowanie w kolumnach i wierszach oraz zamieńmy liczebności na procenty.

In [2]:
proc tabulate data=bib.bank missing;
class wykszt wynik_poprz / descending;
table wykszt all, (wynik_poprz all="Razem")*pctn="%";
run;

Opcja `box=` w argumencie `table` umożliwia dodanie tekstu w kwadracie znajdującym się w lewym górnym rogu tabeli:

In [4]:
proc tabulate data=bib.bank missing;
class wykszt wynik_poprz / descending;
table wykszt all, (wynik_poprz all="Razem")*pctn="%" / box="Podział";
run;

Możemy takżę analizować procenty wierszowe oraz kolumnowe:

In [3]:
proc tabulate data=bib.bank missing;
class wykszt wynik_poprz / descending;
table wykszt all, (wynik_poprz all="Razem")*colpctn="%";
table wykszt all, (wynik_poprz all="Razem")*rowpctn="%";
run;

Utwórzmy tabelę dwuwymiarową także dla zmiennej ilościowej:

In [5]:
proc tabulate data=bib.bank;
class wykszt wynik_poprz;
var czas;
table wykszt, wynik_poprz="Czas rozmowy z klientem"*czas=""*max="" / box="MAX";
run;

## Tablice trzywymiarowe

Dodanie kolejnego przecinka powoduje tworzenie nowych tablic według pierwszej podanej zmiennej:

In [6]:
proc tabulate data=bib.bank;
class wykszt wynik_poprz;
var czas;
table wykszt, czas*(min max), wynik_poprz;
run;

## Zadania

1. Jaki procent stanowią braki danych dla zmiennej rodzaj wykonywanej pracy? (0,64%)
2. Ile procent spośród klientów z wyższym wykształceniem ma kredyt hipoteczny (bez uwzględniania braków danych)? (14,71%)
3. Ile wynosi średni czas rozmowy z bankiem wśród klientów posiadających pożyczkę i pracujących jako technik? (239,18 sekund)